<a href="https://colab.research.google.com/github/Carlos1729/DGL/blob/main/SimpleGCN_Pubmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install watermark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.2 MB/s eta 0:00:00


In [2]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu116/repo.html


Looking in links: https://data.dgl.ai/wheels/cu116/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 MB 1.9 MB/s eta 0:00:00


In [3]:
!pip install  dglgo -f https://data.dgl.ai/wheels-test/repo.html


Looking in links: https://data.dgl.ai/wheels-test/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 966.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 51.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for littleutils: filename=littleutils-0.2.2-py3-none-any.whl size=7029 sha256=ad6ee3e71056b3900c4a150f78826c45679e9e29cd706c579362bddde75f4c3b
  Stored in directory: /root/.cache/pip/wheels/3d/fe/b0/27a9892da57472e538c7452a721a9cf463c

In [4]:
!pip install torchmetrics==0.11.4


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 8.2 MB/s eta 0:00:00


In [5]:
import math
import time

import dgl
import dgl.function as fn

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data import (
    CiteseerGraphDataset,
    CoraGraphDataset,
    PubmedGraphDataset,
    register_data_args,
)
from dgl.nn.pytorch.conv import SGConv

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [6]:
def evaluate(model, g, features, labels, mask):
    model.eval()
    with torch.no_grad():
        logits = model(g, features)[mask]  # only compute the evaluation set
        labels = labels[mask]
        _, indices = torch.max(logits, dim=1)
        correct = torch.sum(indices == labels)
        return correct.item() * 1.0 / len(labels)

In [7]:
data = PubmedGraphDataset()

Extracting file to /root/.dgl/pubmed_35464cad
Finished data loading and preprocessing.
  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done saving data into cached files.


In [12]:
g = data[0]
g = g.int().to("cuda")

In [13]:
features = g.ndata["feat"]
labels = g.ndata["label"]
train_mask = g.ndata["train_mask"]
val_mask = g.ndata["val_mask"]
test_mask = g.ndata["test_mask"]
in_feats = features.shape[1]
n_classes = data.num_labels
n_edges = g.num_edges()

/usr/local/lib/python3.10/dist-packages/dgl/data/utils.py:354: UserWarning: Property dataset.num_labels will be deprecated, please use dataset.num_classes instead.
  warnings.warn(


In [14]:
print(
        """----Data statistics------'
      #Edges %d
      #Classes %d
      #Train samples %d
      #Val samples %d
      #Test samples %d"""
        % (
            n_edges,
            n_classes,
            train_mask.int().sum().item(),
            val_mask.int().sum().item(),
            test_mask.int().sum().item(),
        )
    )

----Data statistics------'
      #Edges 88651
      #Classes 3
      #Train samples 60
      #Val samples 500
      #Test samples 1000


In [18]:
train_mask

tensor([ True,  True,  True,  ..., False, False, False], device='cuda:0')

In [19]:
val_mask

tensor([False, False, False,  ..., False, False, False], device='cuda:0')

In [20]:
test_mask

tensor([False, False, False,  ...,  True,  True,  True], device='cuda:0')

In [23]:
n_edges = g.num_edges()
# add self loop
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

# create SGC model
model = SGConv(in_feats, n_classes, k=2, cached=True, bias=False)
model.cuda()

SGConv(
  (fc): Linear(in_features=500, out_features=3, bias=False)
)

In [25]:
loss_fcn = torch.nn.CrossEntropyLoss()

# use optimizer
optimizer = torch.optim.Adam(
    model.parameters(), lr=0.2, weight_decay=5e-6
)

# initialize graph
dur = []

In [26]:
for epoch in range(100):
    print(epoch)
    model.train()
    if epoch >= 3:
        t0 = time.time()
    # forward
    logits = model(g, features)  # only compute the train set
    loss = loss_fcn(logits[train_mask], labels[train_mask])

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch >= 3:
        dur.append(time.time() - t0)

    acc = evaluate(model, g, features, labels, val_mask)
    print(
        "Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | Accuracy {:.4f} | "
        "ETputs(KTEPS) {:.2f}".format(
            epoch,
            np.mean(dur),
            loss.item(),
            acc,
            n_edges / np.mean(dur) / 1000,
        )
    )

print()
acc = evaluate(model, g, features, labels, test_mask)
print("Test Accuracy {:.4f}".format(acc))


0


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Time(s) nan | Loss 1.0990 | Accuracy 0.7220 | ETputs(KTEPS) nan
1
Epoch 00001 | Time(s) nan | Loss 1.0155 | Accuracy 0.7280 | ETputs(KTEPS) nan
2
Epoch 00002 | Time(s) nan | Loss 0.9386 | Accuracy 0.7320 | ETputs(KTEPS) nan
3
Epoch 00003 | Time(s) 0.0026 | Loss 0.8682 | Accuracy 0.7340 | ETputs(KTEPS) 33561.63
4
Epoch 00004 | Time(s) 0.0026 | Loss 0.8041 | Accuracy 0.7360 | ETputs(KTEPS) 33936.86
5
Epoch 00005 | Time(s) 0.0026 | Loss 0.7458 | Accuracy 0.7380 | ETputs(KTEPS) 34329.04
6
Epoch 00006 | Time(s) 0.0026 | Loss 0.6929 | Accuracy 0.7440 | ETputs(KTEPS) 34055.75
7
Epoch 00007 | Time(s) 0.0026 | Loss 0.6447 | Accuracy 0.7400 | ETputs(KTEPS) 33625.36
8
Epoch 00008 | Time(s) 0.0026 | Loss 0.6010 | Accuracy 0.7440 | ETputs(KTEPS) 33476.01
9
Epoch 00009 | Time(s) 0.0027 | Loss 0.5613 | Accuracy 0.7460 | ETputs(KTEPS) 33412.13
10
Epoch 00010 | Time(s) 0.0027 | Loss 0.5251 | Accuracy 0.7480 | ETputs(KTEPS) 33310.20
11
Epoch 00011 | Time(s) 0.0027 | Loss 0.4922 | Accuracy 